# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [7]:
Orders = pd.read_csv("https://raw.githubusercontent.com/Dee-M123/lab-dataframe-calculation-and-transformation/refs/heads/main/data/Orders/Orders.csv")

In [8]:
Orders.head(4)

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [13]:
customer_agg = Orders.groupby('CustomerID')['amount_spent'].sum().reset_index()

vip_threshold = customer_agg['amount_spent'].quantile(0.90)

preferred_threshold = customer_agg['amount_spent'].quantile(0.70)

vip_customers = customer_agg[customer_agg['amount_spent'] >= vip_threshold]

preferred_customers = customer_agg[
    (customer_agg['amount_spent'] >= preferred_threshold) &
    (customer_agg['amount_spent'] < vip_threshold)]


In [14]:
preferred_customers

,CustomerID,amount_spent
2,12348,1797.24
3,12349,1757.55
5,12352,2506.04
9,12356,2811.43
13,12360,2662.06
...,...,...
4319,18259,2338.60
4320,18260,2643.20
4328,18272,3078.58
4337,18283,2094.88


In [15]:
vip_customers

,CustomerID,amount_spent
0,12346,77183.60
1,12347,4310.00
10,12357,6207.67
12,12359,6372.58
15,12362,5226.23
...,...,...
4292,18223,6484.54
4294,18225,5509.12
4295,18226,5228.05
4298,18229,7276.90


In [21]:
def label_customer(row):
    if row['amount_spent'] >= vip_threshold:
        return 'VIP'
    elif row['amount_spent'] >= preferred_threshold:
        return 'Preferred'
    else:
        return 'Regular'

customer_agg['Customer_Segment'] = customer_agg.apply(label_customer, axis=1)

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [22]:
# Merge the aggregated and labeled data with original dataframe to get countries
customer_data = pd.merge(customer_agg, Orders[['CustomerID', 'Country']], on='CustomerID', how='left')

# Filter VIP customers
vip_customers = customer_data[customer_data['Customer_Segment'] == 'VIP']

In [30]:
vip_counts = vip_customers['Country'].value_counts()
top_country = vip_counts.idxmax()
top_count = vip_counts.max()

print(f"{top_country} with {top_count}")

United Kingdom with 131497


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [32]:
# Merge aggregated/labeled customer data with country info
customer_data = pd.merge(customer_agg, Orders[['CustomerID', 'Country']], on='CustomerID', how='left')

# Filter customers who are VIP or Preferred
vip_pref_customers = customer_data[customer_data['Customer_Segment'].isin(['VIP', 'Preferred'])]

vip_pref_counts = vip_pref_customers['Country'].value_counts()

top_country = vip_pref_counts.idxmax()
top_count = vip_pref_counts.max()

print(f"{top_country} with {top_count}")

United Kingdom with 243879
